# Optimal Portfolio Creation

## Introduction

In this notebook, I will discuss the process of creating optimal risky portfolio from the securities in Dhaka Stock Exchange. This is a course requirements for this semesters Investment and Portfolio Analysis of IBA MBA. The course is taken by Mrs Shakila Yasmin. I will discuss the process from collecting historical data, storing in usable format and preprocessing for analysis to creating investable portfolio. Its will be a wild ride. Hope you all enjoy it.  

## Outlines

- Scrap or collect historical data of all securities in DSE
- Store it MongoDb for easy collection and use
- Create an nodejs api for easy collection to use in Pandas [optional]
- Get Data for specific securities, do a base line analysis and derive decision particulars
- From particular of all the securities sort the best numbers of securities
- Get/Scrap fundamentals data from annual report and store in database and create retrieve api
- Do fundamental analysis of securities to find undervalued and overvalued stocks
    - Divident Discount Model
    - Free Cashflow Model
    - Earnings Multiplier Model
    - Ratio Analysis
- Do technical analysis with prophet, regression and machine learning techniques
- Select the best 5 and make all optimum portfolio combination of two stocks
- Select the best one and allocate funds along with risk free asset based on investor risk aversion
    - Figure out riskfree rate
    - Figure out risk aversion ( score card method )
- Invest and update the process along the way


__Special Requirement__
- Use latex for Report and use versioning
- Make a visualization frontend with vue

## Lets Begin

### Scraping the Data

In [1]:
# importing modules necessary
import pandas as pd
import requests as req
from pymongo import MongoClient
from bs4 import BeautifulSoup
import datetime

In [2]:
client = MongoClient('localhost', 27017)

In [3]:
db = client.dsedb
daily_stock = db.daily_stock

In [4]:
url = "https://www.dsebd.org/day_end_archive.php"
start = "2018-09-02"
end = "2019-09-02"
symbol = "All Instrument"
button = "View Day End Archive"
form = {
    "DayEndSumDate1" : start,
    "DayEndSumDate2" : end,
    "Symbol" : symbol,
    "ViewDayEndArchive" : button
}

In [20]:
html = req.post(url, data = form)
print(html)
source = BeautifulSoup(html.content)

<Response [200]>


In [21]:
table = source.select_one("body > table:nth-child(9) > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > table ")

body > table:nth-child(9) > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > table
/html/body/table[2]/tbody/tr/td[2]/table/tbody/tr[2]/td[1]/table

In [22]:
df = pd.read_html(table.prettify())[0]

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108726 entries, 0 to 108725
Data columns (total 12 columns):
0     108726 non-null object
1     108726 non-null object
2     108726 non-null object
3     108726 non-null object
4     108726 non-null object
5     108726 non-null object
6     108726 non-null object
7     108726 non-null object
8     108726 non-null object
9     108726 non-null object
10    108726 non-null object
11    108726 non-null object
dtypes: object(12)
memory usage: 10.0+ MB


In [63]:
help(df.rename)

Help on method rename in module pandas.core.frame:

rename(mapper=None, index=None, columns=None, axis=None, copy=True, inplace=False, level=None, errors='ignore') method of pandas.core.frame.DataFrame instance
    Alter axes labels.
    
    Function / dict values must be unique (1-to-1). Labels not contained in
    a dict / Series will be left as-is. Extra labels listed don't throw an
    error.
    
    See the :ref:`user guide <basics.rename>` for more.
    
    Parameters
    ----------
    mapper : dict-like or function
        Dict-like or functions transformations to apply to
        that axis' values. Use either ``mapper`` and ``axis`` to
        specify the axis to target with ``mapper``, or ``index`` and
        ``columns``.
    index : dict-like or function
        Alternative to specifying axis (``mapper, axis=0``
        is equivalent to ``index=mapper``).
    columns : dict-like or function
        Alternative to specifying axis (``mapper, axis=1``
        is equivalent 

In [25]:
df = df.rename(df.iloc[0], axis=1)[1:]

In [26]:
df = df.drop('#', axis=1)

In [27]:
df['Date'] = pd.to_datetime(df['DATE'])

In [28]:
df = df.set_index('Date')

In [29]:
df.head()

,DATE,TRADING CODE,LTP*,HIGH,LOW,OPENP*,CLOSEP*,YCP,TRADE,VALUE (mn),VOLUME
Date,,,,,,,,,,,
2019-09-02,2019-09-02,1JANATAMF,4.1,4.2,4,4.1,4.1,4.1,74,2.146,528369
2019-09-01,2019-09-01,1JANATAMF,4.1,4.2,4.1,4.2,4.1,4.2,98,1.471,357297
2019-08-29,2019-08-29,1JANATAMF,4.1,4.3,4.1,4.2,4.2,4.2,115,1.974,469941
2019-08-28,2019-08-28,1JANATAMF,4.2,4.3,4.1,4.3,4.2,4.4,137,1.956,467646
2019-08-27,2019-08-27,1JANATAMF,0,0,0,0,4.4,4.4,0,0,0


In [30]:
df = df.drop('DATE', axis=1)

In [31]:
df.head()

,TRADING CODE,LTP*,HIGH,LOW,OPENP*,CLOSEP*,YCP,TRADE,VALUE (mn),VOLUME
Date,,,,,,,,,,
2019-09-02,1JANATAMF,4.1,4.2,4,4.1,4.1,4.1,74,2.146,528369
2019-09-01,1JANATAMF,4.1,4.2,4.1,4.2,4.1,4.2,98,1.471,357297
2019-08-29,1JANATAMF,4.1,4.3,4.1,4.2,4.2,4.2,115,1.974,469941
2019-08-28,1JANATAMF,4.2,4.3,4.1,4.3,4.2,4.4,137,1.956,467646
2019-08-27,1JANATAMF,0,0,0,0,4.4,4.4,0,0,0


In [32]:
df = df.rename(columns = {'Date': 'DATE'})

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 108725 entries, 2019-09-02 to 2019-01-28
Data columns (total 10 columns):
TRADING CODE    108725 non-null object
LTP*            108725 non-null object
HIGH            108725 non-null object
LOW             108725 non-null object
OPENP*          108725 non-null object
CLOSEP*         108725 non-null object
YCP             108725 non-null object
TRADE           108725 non-null object
VALUE (mn)      108725 non-null object
VOLUME          108725 non-null object
dtypes: object(10)
memory usage: 9.1+ MB


In [34]:
df.head()

,TRADING CODE,LTP*,HIGH,LOW,OPENP*,CLOSEP*,YCP,TRADE,VALUE (mn),VOLUME
Date,,,,,,,,,,
2019-09-02,1JANATAMF,4.1,4.2,4,4.1,4.1,4.1,74,2.146,528369
2019-09-01,1JANATAMF,4.1,4.2,4.1,4.2,4.1,4.2,98,1.471,357297
2019-08-29,1JANATAMF,4.1,4.3,4.1,4.2,4.2,4.2,115,1.974,469941
2019-08-28,1JANATAMF,4.2,4.3,4.1,4.3,4.2,4.4,137,1.956,467646
2019-08-27,1JANATAMF,0,0,0,0,4.4,4.4,0,0,0


In [37]:
help(df.to_dict)

Help on method to_dict in module pandas.core.frame:

to_dict(orient='dict', into=<class 'dict'>) method of pandas.core.frame.DataFrame instance
    Convert the DataFrame to a dictionary.
    
    The type of the key-value pairs can be customized with the parameters
    (see below).
    
    Parameters
    ----------
    orient : str {'dict', 'list', 'series', 'split', 'records', 'index'}
        Determines the type of the values of the dictionary.
    
        - 'dict' (default) : dict like {column -> {index -> value}}
        - 'list' : dict like {column -> [values]}
        - 'series' : dict like {column -> Series(values)}
        - 'split' : dict like
          {'index' -> [index], 'columns' -> [columns], 'data' -> [values]}
        - 'records' : list like
          [{column -> value}, ... , {column -> value}]
        - 'index' : dict like {index -> {column -> value}}
    
        Abbreviations are allowed. `s` indicates `series` and `sp`
        indicates `split`.
    
    into : c

In [38]:
daily_stock.insert_many(df.to_dict('records'))

In [50]:
datetime.date.today().strftime("%Y-%m-%d")

'2019-09-03'

In [53]:
help(pd.date_range)

Help on function date_range in module pandas.core.indexes.datetimes:

date_range(start=None, end=None, periods=None, freq=None, tz=None, normalize=False, name=None, closed=None, **kwargs)
    Return a fixed frequency DatetimeIndex.
    
    Parameters
    ----------
    start : str or datetime-like, optional
        Left bound for generating dates.
    end : str or datetime-like, optional
        Right bound for generating dates.
    periods : integer, optional
        Number of periods to generate.
    freq : str or DateOffset, default 'D'
        Frequency strings can have multiples, e.g. '5H'. See
        :ref:`here <timeseries.offset_aliases>` for a list of
        frequency aliases.
    tz : str or tzinfo, optional
        Time zone name for returning localized DatetimeIndex, for example
        'Asia/Hong_Kong'. By default, the resulting DatetimeIndex is
        timezone-naive.
    normalize : bool, default False
        Normalize start/end dates to midnight before generating dat

In [87]:
today = datetime.date.today().strftime("%Y-%m-%d")
start = "2017-" + today
#dt_range = pd.date_range(start = , freq="M", periods= 25)
print(today.day,start)

AttributeError: 'str' object has no attribute 'day'

In [80]:
dt_range

DatetimeIndex(['2017-08-31', '2017-09-30', '2017-10-31', '2017-11-30',
               '2017-12-31', '2018-01-31', '2018-02-28', '2018-03-31',
               '2018-04-30', '2018-05-31', '2018-06-30', '2018-07-31',
               '2018-08-31', '2018-09-30', '2018-10-31', '2018-11-30',
               '2018-12-31', '2019-01-31', '2019-02-28', '2019-03-31',
               '2019-04-30', '2019-05-31', '2019-06-30', '2019-07-31',
               '2019-08-31'],
              dtype='datetime64[ns]', freq='M')

In [76]:
for i in range(len(dt_range)-1):
    print(dt_range[i].strftime("%Y-%m-%d") + " and " + dt_range[(i+1)].strftime("%Y-%m-%d"))

2017-09-30 and 2017-10-31
2017-10-31 and 2017-11-30
2017-11-30 and 2017-12-31
2017-12-31 and 2018-01-31
2018-01-31 and 2018-02-28
2018-02-28 and 2018-03-31
2018-03-31 and 2018-04-30
2018-04-30 and 2018-05-31
2018-05-31 and 2018-06-30
2018-06-30 and 2018-07-31
2018-07-31 and 2018-08-31
2018-08-31 and 2018-09-30
2018-09-30 and 2018-10-31
2018-10-31 and 2018-11-30
2018-11-30 and 2018-12-31
2018-12-31 and 2019-01-31
2019-01-31 and 2019-02-28
2019-02-28 and 2019-03-31
2019-03-31 and 2019-04-30
2019-04-30 and 2019-05-31
2019-05-31 and 2019-06-30
2019-06-30 and 2019-07-31
2019-07-31 and 2019-08-31


In [91]:
today = datetime.date.today()

In [92]:
today.day

3

In [93]:
today.year - 2

2017

In [101]:
start = str(today.year - 2) + "-" + str(today.month) + "-" + str(today.day)

In [102]:
start

'2017-9-3'

In [104]:
html = BeautifulSoup(req.post(url, data = form).content).select_one("body > table:nth-child(9) > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > table")

KeyboardInterrupt: 